In diesem Skript wird nicht auf ein sequentielles Model (Sequential) gesetzt, sondern auf ein Model

In [1]:
import numpy as np
import tensorflow as tf

from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import MaxPool2D

from sklearn.model_selection import cross_val_score
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV # GridSearch + CrossValidation

from mnistData import MNIST

Train size: 60000
Test size: 10000
Train shape: (60000, 28, 28, 1)
Test shape: (10000, 28, 28, 1)
Min of x_train: 0.0
Max of x_train: 1.0


In [2]:
np.random.seed(0)
tf.random.set_seed(0)

In [3]:
def build_model(
    filters_1: int,
    kernel_size_1: int,
    filters_2: int,
    kernel_size_2: int,
    filters_3: int,
    kernel_size_3: int
) -> Model:
    input_img = Input(shape=(28,28,1))

    x = Conv2D(filters=filters_1, kernel_size=kernel_size_1, padding='same')(input_img)
    x = Activation("relu")(x)
    x = Conv2D(filters=filters_1, kernel_size=kernel_size_1, padding='same')(x)
    x = Activation("relu")(x)
    x = MaxPool2D()(x)

    x = Conv2D(filters=filters_2, kernel_size=kernel_size_2, padding='same')(input_img)
    x = Activation('relu')(x)
    x = Conv2D(filters=filters_2, kernel_size=kernel_size_2, padding='same')(x)
    x = Activation('relu')(x)
    x = MaxPool2D()(x)

    x = Conv2D(filters=filters_3, kernel_size=kernel_size_3, padding='same')(x)
    x = Activation('relu')(x)
    x = Conv2D(filters=filters_3, kernel_size=kernel_size_3, padding='same')(x)
    x = Activation('relu')(x)
    x = MaxPool2D()(x)

    x = Flatten()(x)
    x = Dense(units=128)(x)
    x = Activation("relu")(x)
    x = Dense(units=10)(x)
    y_pred = Activation("softmax")(x)

    model = Model(
        inputs = [input_img],
        outputs = [y_pred]
    )

    model.compile( 
        loss='categorical_crossentropy',
        optimizer='Adam',
        metrics=['accuracy']
    )
    
    model.summary()

    return model

In [4]:
data = MNIST(with_normalization=True)
x_train, y_train = data.get_train_set()

param_grid = {
    'filters_1': [16, 32],
    'kernel_size_1': [3, 5],
    'filters_2': [32, 64],
    'kernel_size_2': [3, 5],
    'filters_3': [64, 128],
    'kernel_size_3': [5]
}

keras_clf = KerasClassifier(
    build_fn=build_model, # Funktionsobjekt übergeben, Fit-Methode ist hier intern gelöst und Parameter müssen angegeben werden
    epochs=3,
    batch_size=128,
    verbose=1
)

grid_cv = GridSearchCV(
    estimator=keras_clf,
    param_grid=param_grid,
    n_jobs=1,
    verbose=0,
    cv=3
)

grid_result = grid_cv.fit(x_train, y_train)

C:\Users\gutsc\AppData\Local\Temp/ipykernel_3196/1731250149.py:13: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead.
  keras_clf = KerasClassifier(


Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 28, 28, 1)]       0         
                                                                 
 conv2d_2 (Conv2D)           (None, 28, 28, 32)        320       
                                                                 
 activation_2 (Activation)   (None, 28, 28, 32)        0         
                                                                 
 conv2d_3 (Conv2D)           (None, 28, 28, 32)        9248      
                                                                 
 activation_3 (Activation)   (None, 28, 28, 32)        0         
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 14, 14, 32)       0         
 2D)                                                             
                                                             

In [5]:
print(f"Best: {grid_result.best_score_} using {grid_result.best_params_}")
means = grid_result.cv_results_["mean_test_score"]
stds = grid_result.cv_results_["std_test_score"]
params = grid_result.cv_results_["params"]

for mean, std, param in zip(means, stds, params):
    print(f"Acc: {mean} (+/- {std * 2}) with: {param}")

Best: 0.9896833300590515 using {'filters_1': 32, 'filters_2': 32, 'filters_3': 128, 'kernel_size_1': 5, 'kernel_size_2': 5, 'kernel_size_3': 5}
Acc: 0.9873499870300293 (+/- 0.0009933077742932473) with: {'filters_1': 16, 'filters_2': 32, 'filters_3': 64, 'kernel_size_1': 3, 'kernel_size_2': 3, 'kernel_size_3': 5}
Acc: 0.9882833361625671 (+/- 0.0009809448570342369) with: {'filters_1': 16, 'filters_2': 32, 'filters_3': 64, 'kernel_size_1': 3, 'kernel_size_2': 5, 'kernel_size_3': 5}
Acc: 0.9871000051498413 (+/- 0.0031633150943912547) with: {'filters_1': 16, 'filters_2': 32, 'filters_3': 64, 'kernel_size_1': 5, 'kernel_size_2': 3, 'kernel_size_3': 5}
Acc: 0.9872666796048483 (+/- 0.0016579728598135972) with: {'filters_1': 16, 'filters_2': 32, 'filters_3': 64, 'kernel_size_1': 5, 'kernel_size_2': 5, 'kernel_size_3': 5}
Acc: 0.9871833523114523 (+/- 0.0021296870004828884) with: {'filters_1': 16, 'filters_2': 32, 'filters_3': 128, 'kernel_size_1': 3, 'kernel_size_2': 3, 'kernel_size_3': 5}
Acc: 